In [1]:
import numpy as np
import pandas as pd

from quantifilib.data.data_loader import YahooFinanceLoader

loader = YahooFinanceLoader()
data = loader.load(
    '^GSPC',
    start = '2000-01-01',
    interval = '1d',
    auto_adjust = False,
    multi_level_index = False,
    progress = False
)

In [2]:
from quantifilib.strategy.price_based.mean_reversion import BollingerBandLabeling

bollinger = BollingerBandLabeling(
    data = data,
    upper_multiplier = 2,
    lower_multiplier = 2,
    window = 20,
    method = 'sma',
    price_col_name = 'Close'
)

In [3]:
bollinger_signal = bollinger.get_labels()

### Set Pipeline up for Meta Labeling

In [6]:
from  quantifilib.strategy.meta_labeling import MetaLabeling

meta = MetaLabeling(
    data = data,
    signal = BollingerBandLabeling(
        data = data,
        upper_multiplier = 2,
        lower_multiplier = 2,
        window = 20,
        method = 'sma',
        price_col_name = 'Close'
    ).get_labels()
)
meta_label = meta.get_meta_label(barrier_days = 5)

In [7]:
meta_label.sort_index()

,bins,meta_label,actual_ret
Date,,,
2000-02-22,1,1,0.010539
2000-03-20,-1,0,0.046154
2000-03-24,-1,1,-0.018907
2000-04-18,1,1,0.013443
2000-07-17,-1,1,-0.030586
...,...,...,...
2025-03-12,1,1,0.013571
2025-04-09,1,0,-0.033206
2025-06-25,-1,0,0.022202


In [9]:
meta_label['meta_label'].value_counts()

meta_label
0    176
1    166
Name: count, dtype: int64

In [10]:
from quantifilib.strategy.price_based.mean_reversion import RSILabeling

meta = MetaLabeling(
    data = data,
    signal = RSILabeling(
        data = data,
        window = 14,
        upper_threshold = 70,
        lower_threshold = 30,
    ).get_labels()
)
meta_label = meta.get_meta_label(barrier_days = 5)

In [12]:
meta_label['meta_label'].value_counts()

meta_label
1    206
0    202
Name: count, dtype: int64